### Unit 1 Homework:  Scraping the Yelp Website

Welcome!  For this homework assignment you'll be tasked with building a web scraper in a manner that builds on what was covered in our web scraping class.

The assignment will extend the lab work done during that time, where we built a dataset that listed the name, number of reviews and price range for restaurant on the following web page: https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1

**What You'll Turn In:**

A finished jupyter notebook that walks us through the steps you took in order to get your results.  Provide notes where appropriate to explain what you are doing.

The notebook should produce a finished dataset at the end.  

If for some reason you're experiencing problems with the final result, please let someone know when turning it in.
 
The homework is divided into five tiers, each of which have increasing levels of difficulty:

##### Tier 1: Five Columns From the First Page

At the most basic level for this assignment, you will need to extend what we did in class, and create a dataset that has five columns in it that are 30 rows long.  This means you will not need to go off the first page in order to complete this section.

##### Tier 2:  100 Row Dataset With At Least 3 Columns

For this portion of the assignment, take 3 of your columns from step 1, and extend them out to multiple pages on the yelp website.  You should appropriately account for the presence of missing values.

##### Tier 3:  100 Row Dataset With At Least 5 Columns

Very similar to Tier 2, but if you use this many columns you will be forced to encounter some columns that will frequently have missing values, whereas with Tier 2 you could likely skip these if you wanted to.  

##### Tier 4:  100 Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Restaurants often have different categories associated with them, so grabbing them individually as separate values is often challenging.  To complete this tier, you'll have to find a way to 'pick out' each of the individual categories as their own separate column value.  

##### Tier 5:  Unlimited Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Take what you did in Tier 4, and extend it so that the code will work with an arbitrary number of pages.  Ie, regardless of how many pages there are listing the best restaurants in London, your scraper will find them, and cleanly parse their information into clean datasets.

### Hints

Here are a few tips that will save you time when completing this assignment:

 - The name, average rating, total ratings and neighborhood of a restaurant tend to be the 'easy' ones, because they rarely have missing values, so what ever logic you use on the first page will typically apply to all pages.  They are a good place to start
 - Phone numbers, price ranges and reviews are more commonly missing, so if you are trying to get a larger number of items from them across multiple pages you should expect to do some error handling
 - You can specify any sort of selector when using the `find_all()` method, not just `class`.  For example, imagine you have the following `<div>` tag:
    `<div class='main-container red-blue-green' role='front-unit' aria-select='left-below'>Some content here</div>`
    
   This means that when you use `scraper.find_all('div')`, you can pass in arguments like `scraper.find_all('div', {'role': 'front-unit'})` or anything else that allows you to isolate that particular tag.
 - When specifying selectors like `{'class': 'dkght__384Ko'}`, sometimes less is more.  If you include multiple selectors, you are saying return a tag with **any one of these** distinctions, not all of them.  So if your results are large, try different combinations of selectors to get the smallest results possible.
 - If you begin dealing with values that are unreliably entered, you should use the 'outside in' technique where you grab a parent container that holds the element and find a way to check to see if a particular value is there by scraping it further.  The best way to do this is to try and find a unique container for every single restaurant.  This means that you will have a reliable parent element for every single restaurant, and within *each of these* you can search for `<p>`, `<a>`, `<div>`, and `<span>` tags and apply further logic.
 - When you get results from `BeautifulSoup`, you will be given data that's denoted as either `bs4.element.Tag` or `bs4.element.ResultSet`.  They are **not the same**.  Critically, you can search a `bs4.element.Tag` for further items, but you cannot do this with a `bs4.element.ResultSet`.  
 
   For example, let's say you grab all of the divs from a page with `scraper.find_all('div')` and save it as the variable `total_divs`.  This means `total_divs` will look somethig like this:  
   
   `[<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>]`
      
   In this case the variable `total_divs` is a result set and there's nothing else you can do to it directly.  However, every item within `total_divs` is a tag, which means you can scrape it further.  
   
   So if you wanted you could write a line like:  `total_paragraphs = [div.find_all('p') for div in total_divs]`, and get the collection of paragraphs within each div.  
   
   If you confuse the two you'll get the following error message:  
   
   `AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?`
 - The values of the different selectors change periodically on yelp, so if your scraper all of a sudden stops working that's probably why.  Ie, if you have a command like `scraper.find_all('div', {'class': '485dk0W__container09'}` that no longer returns results, the class `485dk0W__container09` may now be `r56kW__container14` or something similar.

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1'


In [6]:
#initial setup

yelp_req     = requests.get(url)
scraper      = BeautifulSoup(yelp_req.text)

## Tier 1: Five Columns from the First Page

In [134]:
reviews      = scraper.find_all('span',{'class':'reviewCount__09f24__EUXPN'})
review_count = [int(review.text) for review in reviews]

In [135]:
review_count

[24, 7, 293, 2, 139, 185, 71, 183, 7, 34]

In [16]:
names = scraper.find_all('a',{'class':'css-166la90'})

In [18]:
names = [name.text for name in names]
names = [name for name in names if len(name)>1]

In [19]:
names

['The Mayfair Chippy',
 'Dishoom',
 'Flat Iron',
 'Ffiona’s Restaurant',
 'Dishoom',
 'The Breakfast Club',
 'Restaurant Gordon Ramsay',
 'The Fat Bear',
 'NOPI',
 'Sketch']

In [22]:
price_ranges = scraper.find_all('span',{'class':'css-xtpg8e'})
price_ranges = [price.text for price in price_ranges]

In [23]:
price_ranges

['££', '££', '££', '££', '££', '££', '££££', '££', '£££', '£££']

In [44]:
### Tried to get the number of stars-
#img class <div class='i-stars__09f24__1T6rz i-stars--regular-5__09f24__N5JxY'
#stars = scraper.find_all('span',{'img class':'offscreen__09f24__1VFco'})
#stars = [rating.text for rating in stars]

In [46]:
#stars

In [53]:
quote = scraper.find_all('p',{'class':'css-e81eai'})
quote = [rev.text for rev in quote]

In [56]:
quote

['“One of the best fish ever with the most tasty chips.\n\nAll of the sauce was on point and it really gave loads of flavour”\xa0more',
 '“Hard to find a way to add any higher praise to the restaurant of the decade in London. Great food, great decor, great folks running it, even great bathrooms.…”\xa0more',
 '“Went to London for vacation and stopped by this  place for dinner! We were originally trying to find nandos (first time trying) but we saw this place across…”\xa0more',
 "“Ffiona's is easily my favorite restaurant in London. The whole experience, from the food, to the atmosphere, to Ffiona herself, felt like family/home.\n\nThe…”\xa0more",
 "“I visited Dishoom during my recent London trip (pre-COVID) and can't wait to go back.\nI went for breakfast and got the vegan Bombay; it was the best vegan meal…”\xa0more",
 "“By far one of my most favorite breakfast places in London! If you're in the area I highly recommend making time to go. It's super small inside and its always…”\xa0more

In [62]:
category = scraper.find_all('span',{'class':'css-f7rmk3'})
category = [cat.text for cat in category]

In [64]:
category

['Fish & Chips',
 'Indian',
 'Steakhouses',
 'British',
 'Indian',
 'Coffee & Tea, ',
 'Breakfast & Brunch, ',
 'American (Traditional)',
 'French, ',
 'British',
 'American (Traditional), ',
 'Soul Food, ',
 'Cajun/Creole',
 'Mediterranean',
 'French, ',
 'Modern European, ',
 'Cocktail Bars']

In [102]:
#get link, remove pagination links, concat with main website

main_page = scraper.find_all('a',{'class':'css-166la90','role': None})
main_page = [website.get('href') for website in main_page]
full_link_main = ['www.yahoo.com' + s for s in main_page]

In [103]:
full_link_main

['www.yahoo.com/biz/the-mayfair-chippy-london-2?osq=Restaurants',
 'www.yahoo.com/biz/dishoom-london?osq=Restaurants',
 'www.yahoo.com/biz/flat-iron-london-2?osq=Restaurants',
 'www.yahoo.com/biz/ffionas-restaurant-london?osq=Restaurants',
 'www.yahoo.com/biz/dishoom-london-7?osq=Restaurants',
 'www.yahoo.com/biz/the-breakfast-club-london-2?osq=Restaurants',
 'www.yahoo.com/biz/restaurant-gordon-ramsay-london-3?osq=Restaurants',
 'www.yahoo.com/biz/the-fat-bear-london?osq=Restaurants',
 'www.yahoo.com/biz/nopi-london?osq=Restaurants',
 'www.yahoo.com/biz/sketch-london-2?osq=Restaurants']

In [112]:
#using container method to get phones and addresses

containers = scraper.find_all('div', {'class','container__09f24__21w3G'})
phone = [container.find_all('p')[1].text for container in containers]
phone = [number if number.replace(' ', '').isdigit() else None for number in phone]

In [115]:
phone

['020 7741 2233',
 '020 7420 9320',
 None,
 '020 7937 4152',
 '020 7420 9322',
 '020 7434 2571',
 '020 7352 4441',
 '020 7236 2498',
 '020 7494 9584',
 '020 7659 4500']

In [116]:
#creating dictionary

df_dict ={
    'Name' : names,
    'Price_Range' : price_ranges,
    'Num_Reviews' : review_count,
    'Website' : full_link_main,
    "Phone_Number" : phone,
    #'Top Review' : quote,
}

In [117]:
pd.DataFrame(df_dict)

,Name,Price_Range,Num_Reviews,Website,Phone_Number
0,The Mayfair Chippy,££,282,www.yahoo.com/biz/the-mayfair-chippy-london-2?...,020 7741 2233
1,Dishoom,££,1841,www.yahoo.com/biz/dishoom-london?osq=Restaurants,020 7420 9320
2,Flat Iron,££,380,www.yahoo.com/biz/flat-iron-london-2?osq=Resta...,None
3,Ffiona’s Restaurant,££,267,www.yahoo.com/biz/ffionas-restaurant-london?os...,020 7937 4152
4,Dishoom,££,547,www.yahoo.com/biz/dishoom-london-7?osq=Restaur...,020 7420 9322
5,The Breakfast Club,££,494,www.yahoo.com/biz/the-breakfast-club-london-2?...,020 7434 2571
6,Restaurant Gordon Ramsay,££££,204,www.yahoo.com/biz/restaurant-gordon-ramsay-lon...,020 7352 4441
7,The Fat Bear,££,122,www.yahoo.com/biz/the-fat-bear-london?osq=Rest...,020 7236 2498
8,NOPI,£££,272,www.yahoo.com/biz/nopi-london?osq=Restaurants,020 7494 9584
9,Sketch,£££,826,www.yahoo.com/biz/sketch-london-2?osq=Restaurants,020 7659 4500


## Tier 2: 100 Row Dataset with at least 3 columns

In [171]:
page_range = [0,10,20,30,40,50,60,70,80,90,100,110]

#names = []
#price_ranges = []
review_count = []
full_link_main = []
phone = []

for i in page_range:
    url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1&start={i}'
    req = requests.get(url).text
    scraper = BeautifulSoup(req)
   # names           = scraper.find_all('a',{'class':'css-166la90'})
   # names           = [name.text for name in names]
    #fnames          = [name for name in names if len(name)>1]
    #names += fnames
    #price_ranges    = scraper.find_all('span',{'class':'css-xtpg8e'})
    #fprice_ranges   = [price.text for price in price_ranges]
    #price_ranges += fprice_ranges
    reviews         = scraper.find_all('span',{'class':'reviewCount__09f24__EUXPN'})
    freview_count   = [int(review.text) for review in reviews]
    review_count += freview_count
    main_page       = scraper.find_all('a',{'class':'css-166la90','role': None})
    main_page       = [website.get('href') for website in main_page]
    ffull_link_main = ['www.yahoo.com' + s for s in main_page]
    full_link_main += ffull_link_main
    containers      = scraper.find_all('div', {'class','container__09f24__21w3G'})
    sphone          = [container.find_all('p')[1].text for container in containers]
    fphone          = [number if number.replace(' ', '').isdigit() else None for number in sphone]
    phone += fphone

In [172]:
#freview_count

In [173]:
#creating dictionary

df_dict2 ={
    #'Name' : names,
    #'Price_Range' : price_ranges,
    'Num_Reviews' : review_count,
    'Website' : full_link_main,
    'Phone_Number' : phone,
}

In [174]:
#len(phone)

In [175]:
#YAY 
pd.DataFrame(df_dict2)

,Num_Reviews,Website,Phone_Number
0,282,www.yahoo.com/biz/the-mayfair-chippy-london-2?...,020 7741 2233
1,1840,www.yahoo.com/biz/dishoom-london?osq=Restaurants,020 7420 9320
2,380,www.yahoo.com/biz/flat-iron-london-2?osq=Resta...,None
3,267,www.yahoo.com/biz/ffionas-restaurant-london?os...,020 7937 4152
4,204,www.yahoo.com/biz/restaurant-gordon-ramsay-lon...,020 7352 4441
...,...,...,...
115,140,www.yahoo.com/biz/patara-london?osq=Restaurants,020 7437 1071
116,21,www.yahoo.com/biz/persepolis-london?osq=Restau...,020 7639 8007
117,268,www.yahoo.com/biz/the-table-london-8?osq=Resta...,020 7401 2760
118,6,www.yahoo.com/biz/haidilao-hot-pot-london?osq=...,020 8150 0616
